<a href="https://colab.research.google.com/github/smahensaria/w266-final-project/blob/main/W266_Neural_Narratives_Fine_Tune_GPT_2_DRAFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Fine-tune a GPT-2 Model

Sample Code Credit [gpt-2-simple](https://colab.research.google.com/drive/1VLG8e7YSEwypxU-noRNhsv5dW4NfTGce) by [Max Woolf](https://minimaxir.com/2019/09/howto-gpt2/) 



In [1]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [2]:
!nvidia-smi

Mon Jul 12 02:28:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80/P4 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [3]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 483Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 3.53Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 291Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:14, 33.7Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 309Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 4.15Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 3.89Mit/s]


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [ ]:
gpt2.mount_gdrive()

Mounted at /content/drive


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

Using "VIST_sis_train.txt" generated in [W266-Neural-Narratives-Using-GPT-2-DRAFT.ipynb](https://github.com/smahensaria/w266-final-project/blob/main/W266-Neural-Narratives-Using-GPT-2-DRAFT.ipynb) 

In [ ]:
file_name = "VIST_sis_train.txt" 

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [ ]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:11<00:00, 11.19s/it]


dataset has 2402710 tokens
Training...
[10 | 27.96] loss=3.31 avg=3.31
[20 | 50.10] loss=3.29 avg=3.30
[30 | 72.70] loss=3.22 avg=3.27
[40 | 95.87] loss=3.14 avg=3.24
[50 | 119.68] loss=3.27 avg=3.25
[60 | 143.25] loss=3.21 avg=3.24
[70 | 166.55] loss=3.26 avg=3.24
[80 | 189.97] loss=3.14 avg=3.23
[90 | 213.54] loss=3.14 avg=3.22
[100 | 237.08] loss=3.18 avg=3.21
[110 | 260.56] loss=3.26 avg=3.22
[120 | 284.01] loss=3.04 avg=3.20
[130 | 307.49] loss=3.22 avg=3.20
[140 | 331.03] loss=3.14 avg=3.20
[150 | 354.54] loss=3.15 avg=3.20
[160 | 378.07] loss=2.93 avg=3.18
[170 | 401.61] loss=3.01 avg=3.17
[180 | 425.18] loss=3.11 avg=3.16
[190 | 448.72] loss=3.16 avg=3.16
[200 | 472.25] loss=3.09 avg=3.16
======== SAMPLE 1 ========
 .the band played their first set and we all relaxed .my friend really liked it in the end .my girlfriend was even more so than me .i went to the show to catch my friends dancing as they danced .they were getting pretty silly though so i played a few of my first show

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [ ]:
gpt2.generate(sess, run_name='run1')

[male] and [female] had just gotten married .they had brought in almost all of their friends that night .[male] was the first to arrive and get the wedding started .[female] was the last to arrive .[male] and [female] are the guests and will be watching from the front .[male] and [female] are the guests and will be watching from the back .it was [male] 's wedding day .[male] and [female] were both there .[male] was drunk but happy .[female] was a little drunk and pregnant .they were happy to be together .i invited all of my friends to my wedding .i had a great time with them .everyone was happy with me .we danced the night away .the friends had a great time that day .the woman was having a good time at the wedding .the bride was having her wedding pictures taken .the bride was having fun with her friend .the couple would be together for the wedding .the guests were waiting in the reception area .the bride and groom were so happy to be there .the wedding was going to start and it was a 

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [ ]:
prefix_dii_1 = "Six or seven bottle rockets next to a specialty firework."
prefix_dii_2 = "the burgers are being grilled on the grill."
prefix_dii_3 = "A woman has big forks with meat on them."
prefix_dii_4 = " An older man stands in the entry of a garage, manning his station at the grill."
prefix_dii_5 = " A man is standing in front of some saw horses at dusk."

text_full = prefix_dii_1 + prefix_dii_2 + prefix_dii_3 + prefix_dii_4 + prefix_dii_5

#### Generate text of max length 250

In [ ]:
gen_text_full = gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix = text_full,
              nsamples=5,
              batch_size=5
              )

Six or seven bottle rockets next to a specialty firework.the burgers are being grilled on the grill.A woman has big forks with meat on them. An older man stands in the entry of a garage, manning his station at the grill. A man is standing in front of some saw horses at dusk.[male] [male] was excited to go to the carnival .he started off with a ride on a couple of rides .afterwards he took a picture of a truck .he was really excited to go .he was so excited to see all the animals .he had a great time .there were so many rides and games at the carnival , i had to take a picture .i was excited to see all the animals .the food was really good , i got to ride in one .we took a picture with the trucks on the way home .the park is a great place to go .the kids need to have fun .the carnival is great for everyone .the kids all love to play .the food is really good .the carnival is a great place to enjoy the night .the kids are having a great time at the carnival .the children are having fun at

#### Image Caption 1 with text length = 15

In [ ]:
gen_text_1 = gpt2.generate(sess,
              length=15,
              temperature=0.7,
              prefix = prefix_dii_1,
              nsamples=5,
              batch_size=5
              )

Six or seven bottle rockets next to a specialty firework.i got a lot of cool rockets for the party .they were all different
Six or seven bottle rockets next to a specialty firework.a cool picture of the props we used to show off .the nice red
Six or seven bottle rockets next to a specialty firework.i had a great time at the party last weekend .the food was very
Six or seven bottle rockets next to a specialty firework.a cool ceramics set for the firework show .our first fire
Six or seven bottle rockets next to a specialty firework.the firework was labeled `` jenga '' .i never get tired


#### Image Caption 2 with text length = 15

In [ ]:
gen_text_2 = gpt2.generate(sess,
              length=15,
              temperature=0.7,
              prefix = prefix_dii_2,
              nsamples=5,
              batch_size=5
              )

the burgers are being grilled on the grill.the food is being served and will be eaten .the food is served and
the burgers are being grilled on the grill.the burgers are being grilled on the grill rack .the burgers are being grilled
the burgers are being grilled on the grill.the burger is done for the kids .the kids are having fun playing in
the burgers are being grilled on the grill.the burgers are lined up nicely .the burgers are ready to go .we
the burgers are being grilled on the grill.look at the enthusiasm on their faces .a great family meal and a fun


#### Image Caption 3 with text length = 15

In [ ]:
gen_text_3 = gpt2.generate(sess,
              length=15,
              temperature=0.7,
              prefix = prefix_dii_3,
              nsamples=5,
              batch_size=5
              )

A woman has big forks with meat on them.her husband is taking a picture .the woman is eating the meat .they
A woman has big forks with meat on them.she just has to get a hula hoop .she is so expensive .
A woman has big forks with meat on them.next , she has a large wheelbarrow .the bellies of the
A woman has big forks with meat on them.this is a picture of a fork .this is a picture of a man
A woman has big forks with meat on them.she is very busy .her family is very happy .someone is helping her


#### Image Caption 4 with text length = 15

In [ ]:
gen_text_4 = gpt2.generate(sess,
              length=15,
              temperature=0.7,
              prefix = prefix_dii_4,
              nsamples=5,
              batch_size=5
              )

 An older man stands in the entry of a garage, manning his station at the grill.the man is working on his grill and is checking his workstation .the
 An older man stands in the entry of a garage, manning his station at the grill.tourists are at the grill at the time of the show/i
 An older man stands in the entry of a garage, manning his station at the grill.there was a long line for food at the restaurant .as the sun set
 An older man stands in the entry of a garage, manning his station at the grill.next to the grill , a man stands in the entryway , making sure
 An older man stands in the entry of a garage, manning his station at the grill.the people were really enjoying the evening .we got some new cathedrals


#### Image Caption 5 with text length = 15

In [ ]:
gen_text_5 = gpt2.generate(sess,
              length=15,
              temperature=0.7,
              prefix = prefix_dii_5,
              nsamples=5,
              batch_size=5
              )

 A man is standing in front of some saw horses at dusk.the man has seen a lot of horses .the man looks around .the
 A man is standing in front of some saw horses at dusk.they are all looking at the sun .this horse is quite large .this
 A man is standing in front of some saw horses at dusk.the view from the side is amazing .the horses are lined up to go
 A man is standing in front of some saw horses at dusk.he is looking at the buildings in the distance .this is a view of
 A man is standing in front of some saw horses at dusk.a horse is riding down the street .a man is standing in front of


In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.